# Non-planar ironing script


### Input Data

This script creates a non planar ironing layer on top of a 3d print as a proof of concept. Given there are no rises in the y plane, then ironing can be achieved by splitting a model into rectangles, zigzagging back and forth between 2 y coordinates while incrementing the z and x coordinates.

The input data need is lines of g-code in the following format

```
G1 X0 Y1 Z0 E0  ; FOR Y VALUE
G1 X2 Y2 Z2 E0
G1 X3 Y2 Z3 E0
```

* The value of E does not matter
* All lines should be G1 Commands
* The first line is only used for getting the secondary y value
* all other lines generate a 2d curve and so the printer and follow the curve while oscillating between the 2 y values.


In [21]:
# reads file and outputs data so you can ensure the data is correct

with open('input.gcode') as f:
    lines = f.readlines()
    
    print(lines)

['G1 X95.732 Y95.165 Z4.427 E5.31346\n', 'G1 X109.835 Y104.835 Z3.909 E5.97543\n', 'G1 X109.404 Y104.835 Z3.750 E5.98819\n', 'G1 X108.786 Y104.835 Z3.570 E6.00639\n', 'G1 X107.835 Y104.835 Z3.382 E6.03437\n', 'G1 X106.869 Y104.835 Z3.279 E6.06275\n', 'G1 X106.224 Y104.835 Z3.253 E6.08171\n', 'G1 X105.581 Y104.835 Z3.256 E6.10062\n', 'G1 X104.307 Y104.835 Z3.334 E6.13805\n', 'G1 X103.055 Y104.835 Z3.482 E6.17486\n', 'G1 X99.395 Y104.835 Z4.070 E6.28245\n', 'G1 X98.328 Y104.835 Z4.216 E6.31380\n', 'G1 X96.963 Y104.835 Z4.357 E6.33453\n', 'G1 X95.732 Y104.835 Z4.427 E6.35322\n', 'G1 X95.111 Y104.835 Z4.440 E6.36264\n', 'G1 X93.858 Y104.835 Z4.420 E6.38166\n', 'G1 X93.226 Y104.835 Z4.386 E6.39126\n', 'G1 X92.589 Y104.835 Z4.336 E6.40092\n', 'G1 X91.525 Y104.835 Z4.216 E6.41707\n', 'G1 X90.654 Y104.835 Z4.090 E6.44269\n', 'G1 X90.209 Y104.835 Z4.004 E6.45575\n']


### Finds 2 Y values to bounce between

In [22]:
line = lines[0].split(' ')

y0 = float(line[2].replace('Y' ,''))

line = lines[1].split(' ')
y1 = float(line[2].replace('Y' ,''))
lines.pop(0)

print(y0, y1)


95.165 104.835


### Generates an array of the needed coordinates

Bounce between y while moving along the xz path, using the helper class point, and adding them to an array

In [23]:
from point import Point

grid = []
y = y0

for line in lines:
    y = y0 if(y == y1) else y1


    line = line.split(' ')
    x= (float(line[1].replace('X' ,'')))
    z = (float(line[3].replace('Z' ,'')))

    point = Point(x, y, z, 0)

    grid.append(point)
    

### Generate g code

generates the final g-code for the ironing

In [24]:
import copy

line_gap = 0.01 # adjust for different ironing smoothness
z_offset = 0 # offset/inset from surface

g_code = []


for i in range(len(grid) - 1):
    corner_a = grid[i]
    corner_b = grid[i + 1]

    # Go towards origin preventing scraping issues
    [corner_b, corner_a] = corner_a.smaller(corner_b)

    # find the number of lines needed to move between points using teh specified line_gap
    number_of_zags = abs((corner_a.x-corner_b.x))/line_gap
    z_gap = -((corner_a.z-corner_b.z)/number_of_zags)

    # deep copy start of rectangle
    current = copy.deepcopy(corner_a)
    current.z += z_offset
    

    # zigzag until made it to final point
    while(current.x > corner_b.x):
        g_code.append(copy.deepcopy(current))
        current.x -= line_gap
        current.z += z_gap
        g_code.append(copy.deepcopy(current))
        current.y = corner_a.y if (corner_a.y != current.y ) else corner_b.y

In [25]:
for(i, point) in enumerate(g_code):
    print(point)

G1 X109.835 Y104.835 Z3.909 E0.000
G1 X109.825 Y104.835 Z3.905 E0.000
G1 X109.825 Y95.165 Z3.905 E0.000
G1 X109.815 Y95.165 Z3.902 E0.000
G1 X109.815 Y104.835 Z3.902 E0.000
G1 X109.805 Y104.835 Z3.898 E0.000
G1 X109.805 Y95.165 Z3.898 E0.000
G1 X109.795 Y95.165 Z3.894 E0.000
G1 X109.795 Y104.835 Z3.894 E0.000
G1 X109.785 Y104.835 Z3.891 E0.000
G1 X109.785 Y95.165 Z3.891 E0.000
G1 X109.775 Y95.165 Z3.887 E0.000
G1 X109.775 Y104.835 Z3.887 E0.000
G1 X109.765 Y104.835 Z3.883 E0.000
G1 X109.765 Y95.165 Z3.883 E0.000
G1 X109.755 Y95.165 Z3.879 E0.000
G1 X109.755 Y104.835 Z3.879 E0.000
G1 X109.745 Y104.835 Z3.876 E0.000
G1 X109.745 Y95.165 Z3.876 E0.000
G1 X109.735 Y95.165 Z3.872 E0.000
G1 X109.735 Y104.835 Z3.872 E0.000
G1 X109.725 Y104.835 Z3.868 E0.000
G1 X109.725 Y95.165 Z3.868 E0.000
G1 X109.715 Y95.165 Z3.865 E0.000
G1 X109.715 Y104.835 Z3.865 E0.000
G1 X109.705 Y104.835 Z3.861 E0.000
G1 X109.705 Y95.165 Z3.861 E0.000
G1 X109.695 Y95.165 Z3.857 E0.000
G1 X109.695 Y104.835 Z3.857 E0.000

### Add gcode back into sliced version and view on a g code visualizer